In [ ]:
import pandas as pd
# Load the JSON file into a Pandas DataFrame
df = pd.read_json('/content/drive/MyDrive/features.json', lines=True)
df2 = pd.read_json('/content/drive/MyDrive/final_project_df.json_part-00000-0c304ca6-2ec1-4926-8acf-802f7fd27bf6-c000.json', lines=True)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['text_length', 'summary_length', 'avg_word_length_summary', 'avg_word_length_text','flesch_reading_ease','gunning_fog']] = scaler.fit_transform(df[['text_length', 'summary_length', 'avg_word_length_summary', 'avg_word_length_text','flesch_reading_ease','gunning_fog']])

# Combine text with features
df = pd.concat([df, df2[['text','summary']]], axis=1)

# Perform one-hot encoding on the 'if_weekday' column
df = pd.get_dummies(df, columns=['if_weekday'], drop_first=False)
df


,score,text_length,summary_length,text_neg,text_neu,text_pos,text_compound_score,sum_neg,sum_neu,sum_pos,...,gunning_fog,avg_word_length_summary,avg_word_length_text,subjectivity_summary,subjectivity_text,ttr_text,text,summary,if_weekday_0,if_weekday_1
0,5,0.032000,0.226562,0.262,0.660,0.078,-0.9901,0.000,1.000,0.000,...,0.077630,0.035088,0.116954,0.00,0.590098,0.731034,The scenes in this film can be very disquietin...,This movie needed to be made.,0,1
1,1,0.010926,0.179688,0.294,0.668,0.038,-0.9062,0.000,1.000,0.000,...,0.094613,0.033333,0.196581,0.00,0.352381,0.846154,"Mexican men are macho rapists, gangsters or in...","This is junk, stay away",0,1
2,5,0.065710,0.226562,0.017,0.876,0.107,0.9792,0.000,1.000,0.000,...,0.070559,0.033626,0.120370,0.00,0.471777,0.697917,"Over the past few years, public television has...",A Rock N Roll History Lesson,1,0
3,5,0.040942,0.421875,0.005,0.796,0.199,0.9904,0.000,1.000,0.000,...,0.061593,0.029971,0.099116,0.00,0.501190,0.702020,I recvd this video (DVD version) as a Christma...,A MUST-HAVE video if you grew up in the 50's...,0,1
4,5,0.027451,0.328125,0.018,0.858,0.124,0.9345,0.000,0.762,0.238,...,0.064874,0.033138,0.132835,0.00,0.635000,0.871795,"Wow! When I saw this show on PBS--that was it,...",If You Like DooWop You Gotta Have This DVD,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71999,4,0.027917,0.125000,0.136,0.753,0.111,-0.8794,0.000,1.000,0.000,...,0.062031,0.028509,0.087963,0.00,0.371111,0.687500,I'm a Sam Elliott fan ... and this was a good ...,You Know My Name,1,0
72000,2,0.032155,0.179688,0.160,0.651,0.190,0.7081,0.417,0.238,0.345,...,0.058313,0.043860,0.143590,0.55,0.593452,0.753846,Sam Elliott's portrait of Marshal Tilghman is ...,"Good cast, lousy script",0,1
72001,1,0.002916,0.093750,0.100,0.900,0.000,-0.1695,0.000,1.000,0.000,...,0.033676,0.105263,0.092593,0.00,0.475000,0.944444,This movie is not worth Sam Elliot. I was ver...,Dissapointed,0,1
72002,4,0.027917,0.125000,0.136,0.753,0.111,-0.8794,0.000,1.000,0.000,...,0.062031,0.028509,0.087963,0.00,0.371111,0.687500,I'm a Sam Elliott fan ... and this was a good ...,You Know My Name,1,0


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Tokenization and padding here (assuming 'text' column for simplicity)
# You should replace this with your actual preprocessing steps
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(sequences, maxlen=200)

# Prepare target variable and split the data
y = df['is_helpful'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model
predictions = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Epoch 1/5
721/721 [==============================] - 46s 56ms/step - loss: 0.5157 - accuracy: 0.7392 - val_loss: 0.4751 - val_accuracy: 0.7636
Epoch 2/5
721/721 [==============================] - 19s 26ms/step - loss: 0.4554 - accuracy: 0.7833 - val_loss: 0.4738 - val_accuracy: 0.7681
Epoch 3/5
721/721 [==============================] - 15s 21ms/step - loss: 0.4125 - accuracy: 0.8119 - val_loss: 0.4865 - val_accuracy: 0.7628
Epoch 4/5
721/721 [==============================] - 16s 22ms/step - loss: 0.3718 - accuracy: 0.8354 - val_loss: 0.4974 - val_accuracy: 0.7604
Epoch 5/5
451/451 [==============================] - 4s 7ms/step
Accuracy: 0.77
Precision: 0.69
Recall: 0.53
F1 Score: 0.60


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Tokenization and padding for 'text' and 'summary'
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text'])
tokenizer.fit_on_texts(df['summary'])

text_sequences = tokenizer.texts_to_sequences(df['text'])
text_data = pad_sequences(text_sequences, maxlen=2000)

summary_sequences = tokenizer.texts_to_sequences(df['summary'])
summary_data = pad_sequences(summary_sequences, maxlen=50)

# Prepare target variable and split the data
y = df['is_helpful'].values
X_combined = np.hstack((text_data, summary_data))
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Splitting text data and summary data after train-test split
text_train = X_train[:, :2000]  # Assuming first 200 columns are text data
summary_train = X_train[:, 2000:]  # Assuming next 50 columns are summary data
text_test = X_test[:, :2000]
summary_test = X_test[:, 2000:]

# Building the model
text_input = Input(shape=(2000,))
summary_input = Input(shape=(50,))
text_layer = Embedding(input_dim=5000, output_dim=128)(text_input)
text_layer = LSTM(64, return_sequences=True)(text_layer)
text_layer = Dropout(0.5)(text_layer)
text_layer = LSTM(32)(text_layer)
text_layer = Dropout(0.5)(text_layer)

summary_layer = Embedding(input_dim=5000, output_dim=128)(summary_input)
summary_layer = LSTM(64, return_sequences=True)(summary_layer)
summary_layer = Dropout(0.5)(summary_layer)
summary_layer = LSTM(32)(summary_layer)
summary_layer = Dropout(0.5)(summary_layer)

concat_layer = concatenate([text_layer, summary_layer])
output = Dense(1, activation='sigmoid')(concat_layer)

model = Model(inputs=[text_input, summary_input], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([text_train, summary_train], y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model
predictions = (model.predict([text_test, summary_test]) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Epoch 1/5
721/721 [==============================] - 134s 177ms/step - loss: 0.5635 - accuracy: 0.7076 - val_loss: 0.4822 - val_accuracy: 0.7608
Epoch 2/5
721/721 [==============================] - 98s 136ms/step - loss: 0.4618 - accuracy: 0.7777 - val_loss: 0.4757 - val_accuracy: 0.7669
Epoch 3/5
721/721 [==============================] - 93s 129ms/step - loss: 0.4196 - accuracy: 0.8046 - val_loss: 0.4757 - val_accuracy: 0.7725
Epoch 4/5
721/721 [==============================] - 90s 125ms/step - loss: 0.3800 - accuracy: 0.8284 - val_loss: 0.4852 - val_accuracy: 0.7692
Epoch 5/5
451/451 [==============================] - 22s 46ms/step
Accuracy: 0.78
Precision: 0.67
Recall: 0.60
F1 Score: 0.63


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertConfig
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to encode text data using BERT tokenizer
def encode_text(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="tf")

# Prepare the dataset
df['combined_text'] = df['summary'] + " [SEP] " + df['text']  # Combining summary and text
encoded_texts = [encode_text(text) for text in df['combined_text']]
input_ids = tf.concat([x['input_ids'] for x in encoded_texts], axis=0)
attention_masks = tf.concat([x['attention_mask'] for x in encoded_texts], axis=0)

y = df['is_helpful'].values

# Split the data
X_train_ids, X_test_ids, X_train_masks, X_test_masks, y_train, y_test = train_test_split(input_ids, attention_masks, y, test_size=0.2, random_state=42)

# Load pre-trained BERT model
bert = TFBertModel.from_pretrained('bert-base-uncased')

# Building the model
input_ids = Input(shape=(128,), dtype='int32', name="input_ids")
attention_masks = Input(shape=(128,), dtype='int32', name="attention_masks")
bert_output = bert(input_ids, attention_mask=attention_masks)[1]
output = Dense(1, activation='sigmoid')(bert_output)

model = Model(inputs=[input_ids, attention_masks], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train_ids, X_train_masks], y_train, epochs=3, batch_size=32, validation_split=0.2)

# Evaluate the model
predictions = (model.predict([X_test_ids, X_test_masks]) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

TypeError: ignored